In [ ]:
from ipynb.fs.full.att_stats import AttStats

In [ ]:
class TreeEvaluator:
    def __init__(self):
        pass

In [ ]:
class BenchmarkResult:
    def __init__(self):
        pass

In [ ]:
class Prediction:
    def __init__(self, prediction: AttStats, correct_label: str):
        self.prediction = prediction
        self.correctLabel = correctLabel

In [ ]:
class Stats:
    def __init__(self, class_label):
        self.class_label = class_label